## LangChain with time tool for the Boeing demo
> **Expected Output**: Action: today's date

LangChain agent with time tool.  The tool is defined with a LangChain decorator and registered with the initialized agent.


In [18]:
import os
import requests
import json
import httpx
from openai import OpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain.agents import tool
from langchain.agents import initialize_agent, AgentType, load_tools
from langchain_core.tools import Tool
from langchain.tools import tool
from datetime import date
import warnings
from cfenv import AppEnv

In [19]:
# Load CF environment
env = AppEnv()

# Get bound service "gen-ai-qwen3-ultra"
chat_service = env.get_service(name="gen-ai-qwen3-ultra")
chat_credentials = chat_service.credentials

# Optional: configure custom http client
httpx_client = httpx.Client(verify=False)

# Initialize LLM with credentials from cfenv
llm = ChatOpenAI(
    temperature=0.9,
    model=chat_credentials["model_name"],
    base_url=chat_credentials["api_base"],
    api_key=chat_credentials["api_key"],
    http_client=httpx_client
)

In [20]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())


# list of tools for agent
tools = [time]

In [21]:
# initialize agent with tools and llm
agent= initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

try:
    result = agent("whats the date today?") 
except: 
    print("exception on external access")




> Entering new AgentExecutor chain...
<think>
Okay, the user is asking for today's date. I need to use the time tool to get that information. The instructions say that the time function should be called with an empty string as the input. So I'll structure the JSON blob correctly. Let me make sure I'm following the format exactly. The action is "time" and the action_input is an empty string. Once I get the observation, I can relay the date back to the user. No other tools are needed here. Alright, let's execute that.
</think>

Action:
```
{
  "action": "time",
  "action_input": ""
}
```

Observation: 2025-09-10
Thought:<think>
Okay, the user asked for today's date. I used the time tool, which returned 2025-09-10. I need to present this as the final answer. The format should be "Final Answer: 2025-09-10". Let me check if there's any other step needed, but since the question is straightforward, that's all.
</think>

Final Answer: 2025-09-10

> Finished chain.
